## LLM Function Calling & Agent

#### Seven 白勝文 [sevenbai@gmail.com](mailto://sevenbai@gmail.com)

Function Calling 就像 LLM 跟真實世界的異次元通道，讓 LLM 不只出一張嘴，還知道如何使用工具，當進一步發展成 Agent 時，更是讓 AI 進入【已知用火】的新時代。

### 環境準備

1. 把 .env.sample 改名為 .env
2. 到 https://aistudio.google.com/app/apikey 申請 Google API key
3. 將 Google API key 填入 .env 檔

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GENAI_API_KEY")) # 不知道從哪個版本起，都要把這行加上
model = genai.GenerativeModel("gemini-1.5-pro") # Function calling 建議採用最好的模型

### 一般對話

In [ ]:
prompt = "請告訴我現在的時間"

response = model.generate_content([{
    "role": "user",
    "parts": [{
        "text": prompt
    }]
}])
print(response.candidates[0].content)

<img src="images/function_calling_0.png" width="300">

### 什麼是 Function Calling？

[Gemini API 文件的解釋](https://ai.google.dev/gemini-api/docs/function-calling?hl=zh-tw)

您可以使用函式呼叫定義自訂函式，並提供給生成式 AI 模型。模型不會直接叫用這些函式，而是產生結構化資料輸出，指定函式名稱和建議的引數。此輸出內容可以呼叫外部 API，所產生的 API 輸出內容也可以整合回模型中，以便取得更全面的查詢回應。函式呼叫可讓 LLM 與即時資訊和各種服務 (例如資料庫、客戶關係管理系統和文件存放區) 互動，藉此提升提供相關且符合情境的答案。

<img src="images/function_calling_0.png" height="300">　　　
<img src="images/function_calling_1.png" height="300">

### 使用 Function Calling

In [ ]:
from google.generativeai.types import FunctionDeclaration, Tool

# 首先準備一個 get_time() function 的宣告

# 定義 function
get_time_func = FunctionDeclaration(
    name = "get_time",  # function 名稱
    description = "Used to get current time",   # function 功能
)
# 包裝成 tool
get_time_tool = Tool(
    function_declarations = [get_time_func],
)
# 放進工具箱
toolbox = [get_time_tool]

In [ ]:
# 把 get_time() 實作出來
import time
def get_time():
    return { "time": time.strftime("%I:%M %p") }

In [ ]:
# 再次呼叫 LLM，但這次提供額外的 tools 給它
messages = [{
    "role": "user",
    "parts": [{
        "text": prompt
    }]
}]
response = model.generate_content(
    messages,
    tools = toolbox,
)
content = response.candidates[0].content
print(content)

就算給了 LLM 工具，Function Calling 也只會給出下一步指示，不會動手執行！

### 完整的 function calling 流程

<img src="images/function_calling_2.png" width="600">

#### 走完剩下的流程

In [ ]:
from google.protobuf.struct_pb2 import Struct

# 把 LLM 回覆的 function calling 加入 message
messages.append(content)

# 實際呼叫 function，並把結果加入 message
function_response = Struct()
function_response.update(get_time())
messages.append({
    "role": "function",
    "parts": [{
        "function_response": {
            "name": content.parts[0].function_call.name,
            "response": function_response
        }
    }]
})

# 再次呼叫 LLM
print(f"Current messages: {messages}")
response = model.generate_content(
    messages,
    tools = toolbox,
)
# LLM 會根據原始問題、function calling 結果產生最終答案
content = response.candidates[0].content
print(f"LLM response: {content}")